<a href="https://colab.research.google.com/github/jbloewencolon/The-Demographics-of-Faerun/blob/main/DnD_Dataset_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
import numpy as np
import random

In [27]:
aglarond_data = {
    "Nation_Name": "Aglarond",
    "Government_Type": "Autocratic",
    "Ruler": "none",
    "Demographics": "Human Majority, also inhabited by Half-Elves and Elves",
    "Capital": "Velprintalar",
    "Population": 1270000,
    "Geographical_Details": "Located in the southwestern most part of the Unapproachable East, bordered by the Sea of Fallen Stars and Altumbel on the west and Thay on the east. It has regions like Yuirwood, and Dragonjaw Mountains to the north. It also encompasses the sub-peninsula Altumbel with the capital Spandeliyon.",
    "Economic_Activities": "Fishing, High rate of Piracy",
    "Historical_Details": "Once rumored to be the elven realm of Yuireshanyaar, home to Star Elves. Has significant magic and mystery with powerful figures like the Simbul and Elminster associated with it.",
    "Defense_Details": "Constantly defending against attacks from Thay.",
    "Notes": ""
}

amn_data = {
    "Nation_Name": "Amn",
    "Government_Type": "Plutocracy",
    "Ruler": "Council of Six",
    "Demographics": "",
    "Capital": "Athkatla",
    "Population": 3000000,
    "Geographical_Details": "",
    "Economic_Activities": "Trade, Entrepreneurship",
    "Historical_Details": "Amn is known for its focus on wealth acquisition and its leadership by the Council of Six, which promotes free enterprise. It's a prominent setting in Baldur's Gate II: Shadows of Amn, known for its wealth, trading routes, and as a hub for adventurers.",
    "Defense_Details": "",
    "Notes": "Features prominently in the video game Baldur's Gate II: Shadows of Amn.",
}

calimshan_data = {
    "Nation_Name": "Calimshan",
    "Government_Type": "Autocracy",
    "Ruler": "Ralan el Pesarkhal",
    "Demographics": "",
    "Capital": "Calimport",
    "Population": 5300000,
    "Geographical_Details": "Covers southern corner of Faerûn, includes Calim Desert and Spider Swamp.",
    "Economic_Activities": "Trade, Magic-item crafting",
    "Historical_Details": "Calimshan is characterized by its pursuit of wealth and use of labor-saving magical devices. The Pasha in Calimport rules but with significant autonomy for city viziers.",
    "Defense_Details": "",
    "Notes": "Known for magical research and features in novels by R.A. Salvatore.",
}

chessenta_data = {
    "Nation_Name": "Chessenta",
    "Government_Type": "City-states",
    "Ruler": "none",
    "Demographics": "Primarily inhabited by humans, but also home to halfling, dwarven, half-orc, and lizardfolk enclaves.",
    "Capital": "Cimbar",
    "Population": 3390000,
    "Geographical_Details": "",
    "Economic_Activities": "",
    "Historical_Details": "A country of individual city-states with diverse racial enclaves.",
    "Defense_Details": "",
    "Notes": "",
}
damara_data = {
    "Nation_Name": "Damara",
    "Government_Type": "Monarchy",
    "Ruler": "King Gareth Dragonsbane",
    "Demographics": "Inhabitants are of human, dwarven, halfling, and half-orc blood.",
    "Capital": "Heliogabalus",
    "Population": 1300000,
    "Geographical_Details": "A sparsely populated northern kingdom of Faerûn. Located to the west of Bloodstone Village which is on the border of Damara and Vaasa.",
    "Economic_Activities": "",
    "Historical_Details": "",
    "Defense_Details": "",
    "Notes": "The main city is Heliogabalus, while King Gareth and his court reside in Bloodstone Village. Most inhabitants live in Heliogabalus."
}

dambrath_data = {
    "Nation_Name": "Dambrath",
    "Government_Type": "",
    "Ruler": "",
    "Demographics": "Majority are human peasants, with a ruling class of half-drow (Crinti).",
    "Capital": "",
    "Population": "",
    "Geographical_Details": "Located at the very south of the Shining South on the continent of Faerûn.",
    "Economic_Activities": "",
    "Historical_Details": "The ruling class, the Crinti, maintain close ties and trade with their drow relatives in the Upper Underdark. Tasloi are commonly encountered in this region.",
    "Defense_Details": "",
    "Notes": ""
}

durpar_data = {
    "Nation_Name": "Durpar",
    "Government_Type": "",
    "Ruler": "",
    "Demographics": "",
    "Capital": "",
    "Population": "",
    "Geographical_Details": "Part of the Shining Lands, surrounded by mountains and a sea with restricted access. It borders the dangerous Veldorn to the west.",
    "Economic_Activities": "",
    "Historical_Details": "Follows the teachings of Satama, representing Toril's Siddharta Gautama (Buddha) and has an Indian flavor.",
    "Defense_Details": "",
    "Notes": "Is together with Var the Golden and Estagund, forming a part of the Shining Lands. Faces geographical constraints with limited access to the surrounding sea and proximity to the dangerous Veldorn."
}
evermeet_data = {
    "Nation_Name": "Evermeet",
    "Government_Type": "Monarchy",
    "Ruler": "Queen Amlaruil Moonflower",
    "Demographics": "Mainly inhabited by sun elves and various prominent elven houses including the Durothils.",
    "Capital": "Leuthilspar",
    "Population": "",
    "Geographical_Details": "An island located far west of the continent of Faerûn in the Trackless Sea. It was a destination during the Retreat for elves, especially sun elves.",
    "Economic_Activities": "",
    "Historical_Details": "Evermeet has faced numerous invasion attempts including by the Red Wizards, the Zhentarim, the drow of Faerûn, and various pirates of the Sword Coast and the Trackless Sea, but remained unconquered. Significant events include the 1371 DR attack by forces under the leadership of the drow goddess Lolth and others, and another attack in 1374 DR by the Daemonfey seeking a piece of a legendary artifact. After the Spellplague in 1385 DR, much of the population was transported to the Feywild's version of the isle, leaving it mostly deserted but pristine. Queen Amlaruil disappeared after the Spellplague, and governance shifted to a council of prominent houses. By 1479 DR, it had almost passed into legend.",
    "Defense_Details": "The island boasts numerous defenses including Spelljamming ships and dragons, which have successfully protected it from various invasion attempts.",
    "Notes": "Evermeet was the subject of several publications including the sourcebook 'Elves of Evermeet' by Anthony Pryor and the novel 'Evermeet: Island of Elves' by Elaine Cunningham. It underwent significant changes following the Spellplague, including an increase in isolation and the emergence of a sect of warlocks dedicated to preserving the island as a refuge."
}

rashemen_data = {
    "Nation_Name": "Rashemen",
    "Government_Type": "Magocratic Gynarchy",
    "Ruler": "Iron Lord (real power held by the witches called hathran)",
    "Demographics": "Almost all Humans",
    "Capital": "Immilmar",
    "Population": 650000,
    "Geographical_Details": "An eastern realm of Faerûn, known for its berserker warriors and spirits protecting the land.",
    "Economic_Activities": "",
    "Historical_Details": "Land of the famous Witches of Rashemen.",
    "Defense_Details": "Protected by berserker warriors and spirits.",
    "Notes": ""
}

mulhorand_data = {
    "Nation_Name": "Mulhorand",
    "Government_Type": "Theocracy",
    "Ruler": "Pharaoh Horustep III",
    "Demographics": "Mulhorandi (Egyptians who continue to build pyramids and governed by pharaohs)",
    "Capital": "Skuld, City of Shadows",
    "Population": 5339520,
    "Geographical_Details": "Located on the continent of Faerûn. Has a long-standing enmity with the kingdom of Unther.",
    "Economic_Activities": "",
    "Historical_Details": "Traditional enemy is Unther, a tribe of Mulan who come from ancient Mesopotamia.",
    "Defense_Details": "",
    "Notes": "The Mulhorandi are enslavists."
}

luiren_data = {
    "Nation_Name": "Luiren",
    "Government_Type": "Theocracy",
    "Ruler": "Faran Ferromar",
    "Demographics": "Primarily inhabited by Halflings, with a small population of Humans, Elves, and Half-Elves.",
    "Capital": "Beluir",
    "Population": 840000,
    "Geographical_Details": "Situated to the north of the city of Delzimmer, near the Eastern Shaar.",
    "Economic_Activities": "",
    "Historical_Details": "Land of the Halflings.",
    "Defense_Details": "",
    "Notes": ""
}

impiltur_data = {
    "Nation_Name": "Impiltur",
    "Government_Type": "Monarchy",
    "Ruler": "Queen Sambryl",
    "Demographics": "Mainly inhabited by Humans, but also home to Dwarves and Halflings.",
    "Capital": "Lyrabar",
    "Population": 1200000,
    "Geographical_Details": "",
    "Economic_Activities": "",
    "Historical_Details": "",
    "Defense_Details": "Defended by an army that includes Paladins and a legion of Bronze Dragons.",
    "Notes": ""
}

halruaa_data = {
    "Nation_Name": "Halruaa",
    "Government_Type": "Magocratic Oligarchy",
    "Ruler": "King Zalathorm",
    "Demographics": "A large majority of the population have magical powers.",
    "Capital": "Halarahh",
    "Population": 1675000,
    "Geographical_Details": "Situated on the continent of Faerûn, surrounded by mountain ranges on three sides and the Great Sea to the south. Divided into sections by three rivers meeting at Lake Halruaa in the center.",
    "Economic_Activities": "",
    "Historical_Details": "Created by archwizards foreseeing the fall of Netheril, combines peace and harmony with powerful magic.",
    "Defense_Details": "Constantly threatened by Crinti marauders from Dambrath.",
    "Notes": "A land of wonders, home to a majority of inhabitants with magical powers."
}

sossal_data = {
    "Nation_Name": "Sossal",
    "Government_Type": "Unknown",
    "Ruler": "Unknown",
    "Demographics": "Sossarhim",
    "Capital": "Naupau",
    "Population": "Unknown",
    "Geographical_Details": "A chilling place located between Pelvuria and Yal-Tengri, with a trade city on the Great Ice Sea called Naupau. Inhabitants sometimes travel to Calimshan.",
    "Economic_Activities": "Unknown",
    "Historical_Details": "Unknown",
    "Defense_Details": "Unknown",
    "Notes": ""
}
tethyr_data = {
    "Nation_Name": "Tethyr",
    "Government_Type": "Feudal Monarchy",
    "Ruler": "Queen Zaranda Star Rhindaun, King Haedrak III",
    "Demographics": "Predominantly human, with a significant elven population",
    "Capital": "Darromar",
    "Population": 3750000,
    "Geographical_Details": "A country located on the Sword Coast of Faerûn, home to the Snowflake Mountains, a region known for the Edificant Library and not directly influenced by any nation.",
    "Economic_Activities": "Unknown",
    "Historical_Details": "Once ruled by a royal family who were all killed, leading to a period of fractured governance with various independent entities. Known for the Company of Eight, a group promoting peace.",
    "Defense_Details": "Unknown",
    "Notes": "An ideal setting for adventures due to its lack of centralized government."
}
thay_data = {
    "Nation_Name": "Thay",
    "Government_Type": "Magocracy",
    "Ruler": "Unknown",
    "Demographics": "Unknown",
    "Capital": "Eltabbar",
    "Population": 4900000,
    "Geographical_Details": "Coastal region known as the Priador. Often at war with neighboring regions.",
    "Economic_Activities": "Unknown",
    "Historical_Details": "Home to the dreaded Red Wizards and known for practicing circle magic and slavery. Introduced in the 1988 publication 'Dreams of the Red Wizards'.",
    "Defense_Details": "Constantly warring with its neighbors.",
    "Notes": ""
}
thesk_data = {
    "Nation_Name": "Thesk",
    "Government_Type": "Oligarchy",
    "Ruler": "Unknown",
    "Demographics": "Humans, Gnomes, Orcs",
    "Capital": "Phsant (Unofficial)",
    "Population": 850000,
    "Geographical_Details": "Region close to Great Dale and economically dependent on the city-state of Telflamm and Kara-Tur goods from the far east.",
    "Economic_Activities": "Trade",
    "Historical_Details": "Home to minor kingdoms and city states, including Telflamm, which is influenced by a thieves guild known as the Shadowmasters.",
}
threskel_data = {
    "Nation_Name": "Threskel",
    "Government_Type": "Unknown",
    "Ruler": "Unknown",
    "Demographics": "Unknown",
    "Capital": "Unknown",
    "Population": "Unknown",
    "Geographical_Details": "An impoverished kingdom on the Sea of Fallen Stars, largely a wasteland and a vassal state of Chessenta.",
    "Economic_Activities": "Unknown",
    "Historical_Details": "Formerly ruled by the dracolich Alasklerbanbastos until defeated by Tchazzar in 1479 DR.",
    "Defense_Details": "Unknown",
    "Notes": ""
}
turmish_data = {
    "Nation_Name": "Turmish",
    "Government_Type": "Republic",
    "Ruler": "Unknown",
    "Demographics": "Mainly humans, with populations of dwarves, halflings, elves, gnomes, half-elves, and half-orcs",
    "Capital": "Alaghôn",
    "Population": 1700000,
    "Geographical_Details": "Located in the north of the Vilhon Reach on the continent of Faerûn.",
    "Economic_Activities": "Unknown",
    "Historical_Details": "Unknown",
    "Defense_Details": "Unknown",
    "Notes": ""
}
sembia_data = {
    "Nation_Name": "Sembia",
    "Government_Type": "Plutocracy",
    "Ruler": "Kendrick the Tall",
    "Demographics": "Unknown",
    "Capital": "Ordulin",
    "Population": 2460000,
    "Geographical_Details": "",
    "Economic_Activities": "",
    "Historical_Details": "",
    "Defense_Details": "",
    "Notes": ""
}

semphar_data = {
    "Nation_Name": "Semphar",
    "Government_Type": "Unknown",
    "Ruler": "Unknown",
    "Demographics": "Unknown",
    "Capital": "Dhaztanar",
    "Population": "Unknown",
    "Geographical_Details": "Semphar is the mightiest nation of the Endless Wastes, far from Faerûn and more remote than Thay. It is a theoretical part of the empire of Mulhorand. It also has a significant town called Phelzel.",
    "Economic_Activities": "Unknown",
    "Historical_Details": "",
    "Defense_Details": "",
    "Notes": ""
}
ulgarth_data = {
    "Nation_Name": "Ulgarth",
    "Government_Type": "Unknown",
    "Ruler": "Unknown",
    "Demographics": "Unknown",
    "Capital": "Unknown",
    "Population": "Unknown",
    "Geographical_Details": "Ulgarth is a country located in the Utter East, facing Var the Golden. With the free cities of Parsanic (Zendar, Dekar, Zendum, Tharkar and maybe more), it acts as a buffer between Faerûn and Zakhara.",
    "Economic_Activities": "Unknown",
    "Historical_Details": "Unknown",
    "Defense_Details": "Unknown",
    "Notes": ""
}

unther_data = {
    "Nation_Name": "Unther",
    "Government_Type": "Magocracy",
    "Ruler": "Northern Wizards",
    "Demographics": "Mainly humans but also smaller numbers of dwarves and halflings.",
    "Capital": "Messemprar",
    "Population": 4260000,
    "Geographical_Details": "Resembles the Mesopotamian civilization. Unther was once a great empire but is now in decline.",
    "Economic_Activities": "Unknown",
    "Historical_Details": "Unther, like its long-standing enemy, Mulhorand, was once a great empire, but now seems to be in decline.",
    "Defense_Details": "Unknown",
    "Notes": "The country is currently ruled by the Northern Wizards from the capital of Messemprar, formerly named Unthalass."
}

vaasa_data = {
    "Nation_Name": "Vaasa",
    "Government_Type": "Various hamlets",
    "Ruler": "none",
    "Demographics": "Inhabited by small villages of humans, dwarves, orcs, and half-orcs.",
    "Capital": "none",
    "Population": 145000,
    "Geographical_Details": "Vaasa is a country on the continent of Faerûn, bordering the Pelvuria, the Great Glacier, to the North. It contains regions such as Palishchuk in the Northeastern Corner and the northernmost part near Palishchuk lies Castle Perilous. It is bordered on the west by the Galena mountains.",
    "Economic_Activities": "Unknown",
    "Historical_Details": "It houses Castle Perilous, a remnant of the time of Zhengyi the Witch-King, and is inhabited by a dracolich named Urshula the Black, controlled by Bregan D'aerthe.",
    "Defense_Details": "Unknown",
    "Notes": ""
}

var_the_golden_data = {
    "Nation_Name": "Var the Golden",
    "Government_Type": "Unknown",
    "Ruler": "Unknown",
    "Demographics": "Unknown",
    "Capital": "Unknown",
    "Population": "Unknown",
    "Geographical_Details": "Var the Golden is a peninsula at the far south-east of Faerûn, facing Ulgarth. It is notable for its vast wheat fields.",
    "Economic_Activities": "Agriculture (wheat fields)",
    "Historical_Details": "Follows the teachings of Satama and forms part of the Shining Lands with Durpar and Estagund.",
    "Defense_Details": "Unknown",
    "Notes": ""
}
erlkazar_data = {
    "Nation_Name": "Erlkazar",
    "Government_Type": "Monarchy",
    "Ruler": "King Korox (as of 1366 DR), later influenced/controlled by Saestra Karanok",
    "Demographics": "Mainly humans, detailed demographics unknown",
    "Capital": "Duhlnarim",
    "Population": "Unknown",
    "Geographical_Details": "Once a part of Tethyr, later declared independence to become Erlkazar. The land features peaceful and pastoral villages interspersed with fortified borders. At night, it transforms under the vampire rule, with bandit armies emerging from caverns beneath Ahlarkhem.",
    "Economic_Activities": "Agriculture, trade (although disrupted by bandit activities), previously had good trade arrangements with many countries",
    "Historical_Details": "Originally a wilderness, it was claimed by Tethyr and went through a series of rulers and political turmoil, eventually forming an independent kingdom under King Morkann. It survived several invasions and troubles, including threats from goblin hordes and vampires.",
    "Defense_Details": "Fortified borders built by Morkann, defense forces include Elestam's Crusaders and the Shieldbreaker Ogres (allied during the goblin invasion).",
    "Notes": "The region has a complex history, with periods of wilderness, loyalty struggles, rebellions, and several rulers and governance changes. The land saw significant transformations, including secession from Tethyr and establishment of Erlkazar kingdom."
}
erlkazar_data = {
    "Nation_Name": "Erlkazar",
    "Government_Type": "Monarchy",
    "Ruler": "King Korox Morkann (as of 1366 DR), later influenced by Saestra Karanok",
    "Demographics": "Farmers and hillfolk, significant bandit population",
    "Capital": "Duhlnarim",
    "Population": "Unknown",
    "Geographical_Details": "A land once wilderness, then became a part of Tethyr, later established as Erlkazar with fortified borders, experiences nocturnal bandit raids orchestrated from Ahlarkhem",
    "Economic_Activities": "Trade with various countries, albeit threatened by bandit activities",
    "Historical_Details": "Founded by Karazir Tiiraklar, later seceded from Tethyr under King Morkann; witnessed various political upheavals, battles with goblin hordes and a vampiric influence in later years",
    "Defense_Details": "Fortified borders; defense forces include Elestam's Crusaders and at a time, the Shieldbreaker Ogres",
    "Notes": "Plagued by vampires and dragons; Saestra Karanok manipulated the ruling class and established a bandit army"
}
estagund_data = {
    "Nation_Name": "Estagund",
    "Government_Type": "Unknown",
    "Ruler": "Unknown",
    "Demographics": "Influenced by Indian culture",
    "Capital": "Unknown",
    "Population": "Unknown",
    "Geographical_Details": "Part of the Shining Lands, alongside Durpar and Var the Golden, centered on trading",
    "Economic_Activities": "Trading",
    "Historical_Details": "Unknown",
    "Defense_Details": "Unknown",
    "Notes": "Unknown"
}
imaskar_data = {
    "Nation_Name": "Imaskar",
    "Government_Type": "Initially Monarchy, later transitioned to a non-hereditary government",
    "Ruler": "Initially artificer-kings; later government headed by three officeholders",
    "Demographics": "Initially included slaves from various regions, later descendants formed Deep Imaskar",
    "Capital": "Deep Imaskar (present)",
    "Population": "Tens of millions (at the peak of the empire), current population Unknown",
    "Geographical_Details": "An ancient empire with notable influence in astronomy and planar exploration; later retreated to an underground city, Deep Imaskar",
    "Economic_Activities": "Unknown",
    "Historical_Details": "An overpowering ancient empire; witnessed a cataclysmic slave uprising and later a resurgence from the underground city Deep Imaskar",
    "Defense_Details": "Initially held formidable power, later defense mechanisms are Unknown",
    "Notes": "Involved in inter-planar expeditions, survived multiple upheavals and decided to explore the outside world in recent years"
}
moonshae_isles_data = {
    "Nation_Name": "Moonshae Isles",
    "Government_Type": "Monarchy with different kingdoms",
    "Ruler": "Initially High King Tristan Kendrick, succeeded by High Queen Alicia Kendrick",
    "Demographics": {
        "Ffolk": "Main inhabitants of the Isles, generally monarchical in nature",
        "Northmen": "A viking-like society with violent tendencies, often in conflict with the Ffolk",
        "Firbolgs": "Giant creatures that reside in remote areas and are prone to attacking non-firbolgs",
    },
    "Capital": "Unknown",
    "Population": "Unknown",
    "Geographical_Details": "Comprises various islands with individual kingdoms; trading occurs both overland and overseas",
    "Economic_Activities": {
        "Local_Trade": "Trade between kingdoms on the same island and between various islands",
        "Overseas_Trade": "Trade with external regions including Calimshan, Amn, Tethyr, and Waterdeep",
    },
    "Historical_Details": {
        "Conflicts": {
            "Northmen_vs_Ffolk": "Continuous tension and raids, which have shifted towards conquest in recent years",
            "Peace_Efforts": "Recent efforts by the rulers have brought relative peace between the Northmen and the Ffolk",
        },
        "Magic_and_Religion": "Magic-users are viewed with suspicion; clerics face opposition from druids when spreading their faith",
    },
    "Defense_Details": {
        "Firbolg_Threat": "Firbolgs often attack non-firbolg creatures but mainly reside in remote areas",
        "Other_Threats": "Presence of orcs, goblins, trolls, and other dangerous creatures that pose threats to the inhabitants",
    },
    "Notes": "The region has been witnessing a transition from a conflict-ridden era to a relatively peaceful time, thanks to the efforts of the ruling class. However, the threat from Firbolgs and other creatures remains."
}
lantan_data = {
    "Nation_Name": "Lantan",
    "Government_Type": "Unknown",
    "Ruler": "Unknown",
    "Demographics": {
        "Primary_Race": "Unknown",
        "Other_Races": "Unknown",
    },
    "Capital": "Unknown",
    "Population": "Unknown",
    "Geographical_Details": {
        "Location": "Unknown",
        "Topography": "Unknown",
    },
    "Economic_Activities": {
        "Local_Trade": "Unknown",
        "Overseas_Trade": "Unknown",
    },
    "Historical_Details": {
        "Major_Events": "Unknown",
        "Conflicts": "Unknown",
    },
    "Defense_Details": {
        "Military": "Unknown",
        "Threats": "Unknown",
    },
    "Cultural_Details": {
        "Religion": "Unknown",
        "Magic_and_Technology": "Unknown",
    },
    "Notes": "Additional details about Lantan can be included here."
}
murghom_data = {
    "Nation_Name": "Murghôm",
    "Demographics": {
        "Primary_Race": "Murghômi (Ethnically Mulani)",
        "Language": "Mulhorandi",
        "Religion": "Mulhorandi Pantheon",
    },
    "Political_Details": {
        "Government_Type": "Semiautonomous communities",
        "Leadership": "Ataman (and previously Bey)",
        "Relationship_with_Mulhorand": "Daughter State; Subject territory of Mulhorand",
    },
    "Cultural_Details": {
        "Inspiration": [
            "Cossack Culture",
            "Turanians (Robert E. Howard's Hyborian Age)",
            "Egyptian Influence",
        ],
        "Famous_Personalities": ["Richard Baker"],
        "Skills_and_Trades": ["Horse-riding"],
    },
    "Historical_Details": {
        "Formation": "Series of semiautonomous communities uniting in times of war",
    },
    "Notes": "The area has multiple cultural influences and doesn't strictly map to any real-world equivalent."
}

narfell_data = {
    "Nation_Name": "Narfell",
    "Geographical_Details": {
        "Location": "Eastern part of Faerûn (Cold Lands Region)",
        "Notable_Landmarks": ["Giantspire Mountains", "Lake Ashane", "Icerim Mountains"],
    },
    "Historical_Details": {
        "Rise_and_Fall_of_Empire": {
            "Initial_Rule": "Evil priests with demonic pacts",
            "Expansion": "From Giantspire Mountains to Lake Ashane",
            "Collapse": "Destruction in war with Raumathar around −160 DR",
        },
        "Modern_Times": {
            "Current_State": "Land of barbarian tribes with little knowledge of their history",
            "Economy": "Minimal trade through the Long Road",
        },
    },
    "Cultural_Details": {
        "Annual_Festival": "Bildoobaris",
    },
    "Locations": {
        "Bildoobaris": "Unofficial capital, location for the annual festival",
        "Dun-Tharos": "Ruined former imperial capital",
        "Fortress_Narder": "Ruined fortress, currently a site of sickness",
        "Fortress_of_the_Half-Demon": "Ruins with mysterious stone near Lake Ashane",
        "Mount_Nar": "Snow-capped peak visible from N'Jast",
        "Shaundaular": "Ruined city by the shores of Lake Ashane",
        "Val_Murthag": "Ruined spiritual center of demon worship",
    },
}

ruathym_data = {
    "Nation_Name": "Ruathym",
    "Demographics": {
        "Primary_Race": "Northmen and a seagoing breed of dwarves",
        "Characteristics": "Bold, proud, lusty, and violent",
    },
    "Geographical_Details": {
        "Topography": "Rocky land with deep fjords, clear bays, and magnificent pines",
        "Climate": "Cold with howling storms or chilling fogs",
    },
    "Political_Details": {
        "Government_Type": "Unified nation",
        "Leadership": "First Axe Aumark Lithyl",
    },
    "Historical_Details": {
        "Ancestral_Link": "Ancestral home of the Northmen who settled in Moonshae Isles",
        "Recent_Events": "Unification during the war with Luskan in 1357 DR",
    },
    "Cultural_Details": {
        "Lifestyle": "Ranches and farms, few forests, prevalent war raiding",
    },
    "Notes": "The land and its people are known for their rugged and harsh nature.",
}




In [ ]:
data = [
    {
        "City_Name": "Aglarond",
        "Government_Type": "Autocratic",
        "Ruler": "none",
        "Demographics": {
            "Primary": "Humans",
            "Secondary": ["Half-Elves", "Elves"]
        },
        "Capital": "Velprintalar",
        "Population": 1270000,
        "Geographical_Details": {
            "Location": "southwestern Unapproachable East",
            "Borders": ["Sea of Fallen Stars", "Altumbel", "Thay"],
            "Regions": ["Yuirwood", "Dragonjaw Mountains"],
            "Sub_Peninsula": "Altumbel",
            "Capital_Sub_Peninsula": "Spandeliyon"
        },
        "Economic_Activities": ["Fishing", "Piracy"],
        "Historical_Details": {
            "Past": "Elven realm of Yuireshanyaar",
            "Figures_Associated": ["Simbul", "Elminster"]
        },
        "Defense_Details": "Thay attacks",
        "Notes": ""
    },
        {
        "City_Name": "Thay",
        "Government_Type": "Magocratic",
        "Ruler": "Szass Tam",
        "Demographics": {
            "Primary": "Humans",
            "Secondary": ["Undead", "Others"]
        },
        "Capital": "Eltabbar",
        "Population": 1593000,
        "Geographical_Details": {
            "Location": "east of the Sea of Fallen Stars",
            "Borders": ["Aglarond", "Rashemen", "Mulhorand"],
            "Regions": ["Thayan Plateau", "Sunset Mountains"],
            "Sub_Peninsula": None,
            "Capital_Sub_Peninsula": None
        },
        "Economic_Activities": ["Slavery", "Magic item creation"],
        "Historical_Details": {
            "Past": "A part of Mulhorand",
            "Figures_Associated": ["Szass Tam", "Eltab"]
        },
        "Defense_Details": "Red Wizards",
        "Notes": ""
    },
    {
        "City_Name": "Altumbel",
        "Government_Type": "Confederation",
        "Ruler": "Various City States",
        "Demographics": {
            "Primary": "Humans",
            "Secondary": []
        },
        "Capital": None,
        "Population": 230000,
        "Geographical_Details": {
            "Location": "west of Aglarond",
            "Borders": ["Sea of Fallen Stars"],
            "Regions": [],
            "Sub_Peninsula": "Altumbel",
            "Capital_Sub_Peninsula": None
        },
        "Economic_Activities": ["Fishing"],
        "Historical_Details": {
            "Past": "Aglarondian Territory",
            "Figures_Associated": []
        },
        "Defense_Details": "City-State Militias",
        "Notes": ""
    }
    {
    "City_Name": "Waterdeep",
    "Government_Type": "Oligarchic Republic",
    "Ruler": "The Lords of Waterdeep",
    "Demographics": {
        "Primary": "Humans",
        "Secondary": ["Dwarves", "Elves", "Halflings", "Others"]
    },
    "Capital": None,
    "Population": 2000000,
    "Geographical_Details": {
        "Location": "Coast of the Sea of Swords",
        "Borders": ["Sea of Swords", "Sword Mountains"],
        "Regions": ["Sword Coast"],
        "Sub_Peninsula": None,
        "Capital_Sub_Peninsula": None
    },
    "Economic_Activities": ["Trade", "Craftsmanship", "Magic"],
    "Historical_Details": {
        "Past": "Part of the fallen kingdom of Illefarn",
        "Figures_Associated": ["Khelben 'Blackstaff' Arunsun", "Piergeiron Paladinson"]
    },
    "Defense_Details": "City Watch, City Guard, Griffon Cavalry",
    "Notes": ""
},
{
    "City_Name": "Baldur's Gate",
    "Government_Type": "Oligarchy",
    "Ruler": "The Grand Dukes",
    "Demographics": {
        "Primary": "Humans",
        "Secondary": ["Dwarves", "Elves", "Half-Elves", "Halflings"]
    },
    "Capital": None,
    "Population": 1250000,
    "Geographical_Details": {
        "Location": "Coast of the Sea of Swords",
        "Borders": ["River Chionthar"],
        "Regions": ["Sword Coast"],
        "Sub_Peninsula": None,
        "Capital_Sub_Peninsula": None
    },
    "Economic_Activities": ["Trade", "Manufacturing"],
    "Historical_Details": {
        "Past": "Established as a mercantile outpost",
        "Figures_Associated": ["Balduran", "Eltan", "Skie Silvershield"]
    },
    "Defense_Details": "Flaming Fist, Watch",
    "Notes": ""
}

]



In [14]:
geography = {
    "North": {
        "Temperature_Range": (-30, 0),
        "Cities": {
            "Asgard": {"Nations": ["Kingdom of the North", "Mage Republic"], "Population": 50000, "Average Temperature": -10, "Rainfall": 500},
            "Gotham": {"Nations": ["Elven Empire"], "Population": 80000, "Average Temperature": -5, "Rainfall": 600}
        }
    },
    "South": {
        "Temperature_Range": (0, 50),
        "Cities": {
            "Rivendell": {"Nations": ["Elven Empire", "Dwarven Federation"], "Population": 60000, "Average Temperature": 30, "Rainfall": 1500},
            "Wakanda": {"Nations": ["Mage Republic", "Orcish Tribes"], "Population": 70000, "Average Temperature": 40, "Rainfall": 1200}
        }
    },
    # ... (Add entries for East, West, and Central as well)
}


In [16]:
regions = list(set([
    "Chult", "Galena Mountains", "Great Glacier", "Great Rift", "Oldonnar", "Smoking Mountains",
    "Spine of the World", "Turmish", "Underdark (Darklands)", "Underdark (Earthroot)",
    "Underdark (Northdark)", "Underdark (Old Shanatar)", "Waterdeep", "Chondalwood", "Cormanthor",
    "Elf, Star", "Elven Court", "Evereska", "Evermeet", "Forest of Amtar", "Forest of Lethyr",
    "High Forest", "Menzoberranyr", "Misty Vale", "Outer Sea, The", "Sildeyuir", "Silverymoon",
    "Snow Eagle Aerie", "Wealdath, The", "Yuirwood, The", "Great Dale", "Lantan", "Rathgaunt Hills",
    "Western Heartlands", "Aglarond", "Dalelands", "Dambrath", "Amn", "Calimshan", "Channath Vale",
    "Luiren", "North, The", "Altumbel", "Anauroch", "Chessenta", "Cormyr", "Damara", "Dragon Coast",
    "Endless Wastes", "Golden Water", "Halruaa", "Hordelands", "Impiltur", "Lake of Mists", "Lake of Steam",
    "Lapaliiya", "Moonsea", "Moonshae Isles", "Mulhorand", "Narfell", "Nelanther Isles", "Rashemen",
    "Ride, The", "Samarach", "Sembia", "Shaar, The", "Shadovar", "Shou Expatriate", "Swagdar",
    "Sword Coast, The", "Tashalar", "Tethyr", "Tharsult", "Thay", "Thesk", "Thindol", "Unther",
    "Uthgardt Tribesfolk", "Vaasa", "Vast, The", "Vilhon Reach", "Wizard's Reach"
]))

dwarf_regions = [
    "Chult", "Galena Mountains", "Great Glacier", "Great Rift", "Oldonnar", "Smoking Mountains",
    "Spine of the World", "Turmish", "Underdark (Darklands)", "Underdark (Earthroot)",
    "Underdark (Northdark)", "Underdark (Old Shanatar)", "Waterdeep"
]

elf_regions = [
    "Chondalwood", "Cormanthor", "Elf, Star", "Elven Court", "Evereska", "Evermeet",
    "Forest of Amtar", "Forest of Lethyr", "High Forest", "Menzoberranyr", "Misty Vale",
    "Outer Sea, The", "Sildeyuir", "Silverymoon", "Snow Eagle Aerie", "Waterdeep",
    "Wealdath, The", "Yuirwood, The"
]

gnome_regions = [
    "Great Dale", "Lantan", "Rathgaunt Hills", "Underdark (Northdark)", "Western Heartlands"
]

half_elf_regions = [
    "Aglarond", "Dalelands", "Dambrath", "High Forest", "Silverymoon", "Waterdeep"
]

halfling_regions = [
    "Amn", "Calimshan", "Channath Vale", "Luiren", "North, The", "Western Heartlands"
]

human_regions = [
    "Aglarond", "Altumbel", "Amn", "Anauroch", "Calimshan", "Chessenta", "Chult",
    "Cormyr", "Dalelands", "Damara", "Dambrath", "Dragon Coast", "Endless Wastes",
    "Golden Water", "Great Dale", "Great Glacier", "Halruaa", "Hordelands", "Impiltur",
    "Lake of Mists", "Lake of Steam", "Lantan", "Lapaliiya", "Moonsea", "Moonshae Isles",
    "Mulhorand", "Narfell", "Nelanther Isles", "North, The", "Rashemen", "Ride, The",
    "Samarach", "Sembia", "Shaar, The", "Shadovar", "Shou Expatriate", "Silverymoon",
    "Swagdar", "Sword Coast, The", "Tashalar", "Tethyr", "Tharsult", "Thay", "Thesk",
    "Thindol", "Turmish", "Underdark (Old Shanatar)", "Unther", "Uthgardt Tribesfolk",
    "Vaasa", "Vast, The", "Vilhon Reach", "Waterdeep", "Western Heartlands", "Wizard's Reach"
]

In [17]:
# Initialize the likelihood dictionary
race_region_likelihood = {region: {"Dwarf": 0, "Elf": 0, "Gnome": 0, "Half-Elf": 0, "Halfling": 0, "Human": 0} for region in regions}

# Increment the counts based on the regions lists
for region in dwarf_regions:
    race_region_likelihood[region]["Dwarf"] += 1

for region in elf_regions:
    race_region_likelihood[region]["Elf"] += 1

for region in gnome_regions:
    race_region_likelihood[region]["Gnome"] += 1

for region in half_elf_regions:
    race_region_likelihood[region]["Half-Elf"] += 1

for region in halfling_regions:
    race_region_likelihood[region]["Halfling"] += 1

for region in human_regions:
    race_region_likelihood[region]["Human"] += 1

# Calculate the likelihood
total_regions = len(regions)
for region in regions:
    for race in race_region_likelihood[region]:
        race_region_likelihood[region][race] /= total_regions

In [22]:
race_region_mapping = {
    "Dwarf": dwarf_regions,
    "Elf": elf_regions,
    "Gnome": gnome_regions,
    "Half-Elf": half_elf_regions,
    "Halfling": halfling_regions,
    "Human": human_regions
}

# In your generate_random_data function
def generate_random_data():
    race = random.choice(list(race_region_mapping.keys()))
    region = random.choice(race_region_mapping[race])
    city_key = random.choice(list(region_data['Cities'].keys()))
    city_data = region_data['Cities'][city_key]

    population = city_data["Population"]
    employment_trade = np.random.uniform(0, 100)
    employment_peasant = np.random.uniform(0, 100 - employment_trade)
    employment_noble = 100 - (employment_trade + employment_peasant)

    average_temperature = city_data["Average Temperature"]
    rainfall = city_data["Rainfall"]
    primary_cause_of_death = random.choice(primary_causes_of_death)
    if rainfall < 800:
        primary_cause_of_death = random.choice(["Famine", "War", "Disease"])

    magic_school_graduates = np.random.randint(0, 101)
    wizard_caused_magic = np.random.randint(0, magic_school_graduates + 1) if magic_school_graduates > 0 else 0

    known_gold = employment_noble * 10000
    dragon_sightings = np.random.randint(0, int(known_gold / 10000) + 1)
    danger_level = np.random.randint(1, 21) + (dragon_sightings + wizard_caused_magic) // 2

    return {
        "Region": region_key,
        "City": city_key,
        "Nation": ", ".join(city_data["Nations"]),
        "Races": random.sample(races, np.random.randint(1, len(races) + 1)),
        "Population": population,
        "Average Temperature": average_temperature,
        "Danger Level": danger_level,
        "Primary Cause of Death": primary_cause_of_death,
        "Median Age": np.random.randint(20, 81) - (danger_level // 2),
        "Rainfall": rainfall,
        "Percentage of Homeowners": np.random.uniform(20, 100),
        "Average Rent": np.random.randint(500, 5001),
        "Debt by Household": np.random.randint(0, 100001),
        "Amount of Known Gold in Region": known_gold,
        "Possible Gold in Region": np.random.uniform(0, 1000000),
        "Magic School Graduates": magic_school_graduates,
        "Dragon Sightings": dragon_sightings,
        "Environmental Magic": np.random.randint(0, 101),
        "Wizard Caused Magic": wizard_caused_magic,
        "Likelihood of Getting Cursed": np.random.randint(0, 101),
        "Political Climate": random.choice(political_climates),
        "Openness to Strangers": random.choice(openness_to_strangers),
        "# of Humans": np.random.randint(0, population + 1),
        "Employment %: Trade": employment_trade,
        "Employment %: Peasant": employment_peasant,
        "Employment %: Noble": employment_noble
    }


In [23]:
rows_list = [generate_random_data() for _ in range(1000)]
df = pd.DataFrame(rows_list)


NameError: ignored

In [20]:
df.to_excel('/content/drive/MyDrive/Colab Notebooks/DnD/fictional_data.xlsx', index=False)


In [21]:
df.head()

,Region,City,Nation,Races,Population,Average Temperature,Danger Level,Primary Cause of Death,Median Age,Rainfall,...,Dragon Sightings,Environmental Magic,Wizard Caused Magic,Likelihood of Getting Cursed,Political Climate,Openness to Strangers,# of Humans,Employment %: Trade,Employment %: Peasant,Employment %: Noble
0,South,Rivendell,"Elven Empire, Dwarven Federation",[Mages],60000,30,30,Famine,42,1500,...,2,90,37,9,Stable,Welcoming,48860,38.341756,56.963627,4.694617
1,North,Gotham,Elven Empire,"[Mages, Dwarves, Elves, Orcs]",80000,-5,30,Famine,13,600,...,1,65,21,37,Chaotic,Suspicious,42999,90.082594,7.501114,2.416292
2,South,Rivendell,"Elven Empire, Dwarven Federation","[Humans, Elves, Mages]",60000,30,50,Famine,34,1500,...,6,74,66,13,Unstable,Hostile,32134,69.328428,15.946004,14.725567
3,North,Gotham,Elven Empire,"[Dwarves, Mages, Humans, Elves]",80000,-5,37,Disease,32,600,...,3,4,39,83,Peaceful,Welcoming,2620,52.707539,40.921420,6.371041
4,North,Gotham,Elven Empire,"[Humans, Elves]",80000,-5,27,War,48,600,...,28,74,11,29,Stable,Neutral,53743,27.753869,40.449956,31.796175
